In [25]:
import onnx

from deeplc import DeepLC
import tensorflow as tf
import tf2onnx

In [11]:
dlc = DeepLC()
keras_model = dlc.model
keras_model

['D:\\OneDrive - UGent\\Python Codes\\3Projects\\DeepLC_ONNX\\deeplc\\mods/full_hc_PXD005573_pub_1fd8363d9af9dcad3be7553c39396960.keras']

In [14]:
model = tf.keras.models.load_model(keras_model[0])
model.input_shape

[(None, 60, 6), (None, 30, 6), (None, 55), (None, 60, 20)]

In [23]:
onnx_model_path = "deeplc.onnx"
spec = [
    tf.TensorSpec([None, 60, 6], tf.float32, name="input_1"),
    tf.TensorSpec([None, 30, 6], tf.float32, name="input_2"),
    tf.TensorSpec([None, 55], tf.float32, name="input_3"),
    tf.TensorSpec([None, 60, 20], tf.float32, name="input_4"),
]

onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=onnx_model_path)


rewriter <function rewrite_constant_fold at 0x0000021162D77E20>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


In [32]:
from onnx2torch import convert
import torch

onnx_model = onnx.load(onnx_model_path)
pytorch_onnx_model = convert(onnx_model)

In [39]:
input_tensors = (
    torch.randn(1, 60, 6),  # Input 1
    torch.randn(1, 30, 6),  # Input 2
    torch.randn(1, 55),     # Input 3
    torch.randn(1, 60, 20)  # Input 4
)

# Run PyTorch model
output = pytorch_onnx_model(*input_tensors)
print("PyTorch Model Output:", output)

PyTorch Model Output: tensor([[-14.6315]], grad_fn=<AddBackward0>)
